# PaddlePaddle实现线性回归——房价预测

欢迎大家来到这次实验！在本次实验中我们将使用PaddlePaddle来搭建一个简单的线性回归模型，并利用这一模型预测你的储蓄（在某地区）可以购买多大面积的房子。并且在学习模型搭建的过程中，了解到机器学习的若干重要概念，掌握一个机器学习预测的基本流程。

** 你将学会 **

- 机器学习基本概念：假设函数、损失函数、优化算法
- 数据归一化处理
- 搭建全连接神经网络，实现线性回归

# 背景知识

在实验开始之前，我们先对机器学习的相关概念进行复习和回顾吧！

** 机器学习回顾 **

输入训练数据，利用特定的机器学习方法建立估计函数。得到函数后向这一模型输入测试数据，函数有能力对没有见过的数据进行正确估计，这就是机器学习的过程。
<img src="images/ML_process.png" style="width:300px;height:180px;">

** 线性回归的基本概念 **

线性回归是机器学习中最简单也是最重要的模型之一，其模型建立同样遵循上图流程：获取数据、数据预处理、训练模型、应用模型。

回归模型可以理解为：存在一个点集，用一条曲线去拟合它分布的过程。如果拟合曲线是一条直线，则称为线性回归。如果是一条二次曲线，则被称为二次回归。线性回归是回归模型中最简单的一种。

在线性回归中有几个基本的概念需要掌握：
- 假设函数（Hypothesis Function）
- 损失函数（Loss Function）
- 优化算法（Optimization Algorithm）

###### 假设函数：

假设函数是指，用数学的方法描述自变量和因变量之间的关系，它们之间可以是一个线性函数或非线性函数。
在本次线性回顾模型中，我们的假设函数为 $ \hat{Y}= aX_1+b $ ，其中，$\hat{Y}$表示模型的预测结果（预测房价），用来和真实的Y区分。模型要学习的参数即：a,b。

###### 损失函数：

损失函数是指，用数学的方法衡量假设函数预测结果与真实值之间的误差。这个差距越小预测越准确，而算法的任务就是使这个差距越来越小。

建立模型后，我们需要给模型一个优化目标，使得学到的参数能够让预测值$\hat{Y}$尽可能地接近真实值Y。输入任意一个数据样本的目标值$y_i$和模型给出的预测值$\hat{Y_i}$，损失函数输出一个非负的实值。这个实值通常用来反映模型误差的大小。

对于线性模型来讲，最常用的损失函数就是均方误差（Mean Squared Error， MSE）。
$$MSE=\frac{1}{n}\sum_{i=1}^{n}(\hat{Y_i}-Y_i)^2$$
即对于一个大小为n的测试集，MSE是n个数据预测结果误差平方的均值。

###### 优化算法：

在模型训练中优化算法也是至关重要的，它决定了一个模型的精度和运算速度。本章的线性回归实例中主要使用了梯度下降法进行优化。

**梯度下降**是深度学习中非常重要的概念，值得庆幸的是它也十分容易理解。损失函数$J(w,b)$可以理解为变量$w$和$b$的函数。观察下图，垂直轴表示损失函数的值，两个水平轴分别表示变量$w$和$b$。实际上，可能是更高维的向量，但是为了方便说明，在这里假设$w$和$b$都是一个实数。算法的最终目标是找到损失函数的最小值。而这个寻找过程就是不断地微调变量$w$和$b$的值，一步一步地试出这个最小值。而试的方法就是沿着梯度方向逐步移动。本例中让图中的圆点表示损失函数的某个值，那么梯度下降就是让圆点沿着曲面下降，直到取到最小值或逼近最小值。

因为是凸函数，所以无论初始化在曲面上的哪一点，最终都会收敛到同一点或者相近的点。

<img src="images/gradient_descent.png" style="width:380px;height:180px;">


现在，让我们正式进入实验吧！
本次实验分流程如下：

1. 引用库的导入
2. 数据预处理
3. 构造数据读取的Reader
4. 模型训练
5. 总结

# 1 - 引用库

**首先载入需要用到的库**，它们分别是：

- numpy：一个python的基本库，用于科学计算
- paddle.v2：PaddlePaddle深度学习框架
- matplotlib.use('Agg'):用于在linux系统下运行代码，因为linux终端没有GUI，若是windows，则不需要
- matplotlib.pyplot：用于生成图，在验证模型准确率和展示成本变化趋势时会使用到
- pandas:一种基于NumPy的工具，高效处理数据
- from __future__ import print_function:在开头加上from __future__ import print_function这句之后，即使在python2.X，使用print就得像   python3.X那样加括号使用.

In [1]:
import numpy as np
import paddle.v2 as paddle
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
from __future__ import print_function

# 2 - 数据预处理

**本部分主要完成数据的归一化和数据集的分割操作**

数据集使用的是链家公司上海某地区的房价数据。为了简化模型，假设影响房价的因素只有房屋面积，因此数据集只有两列，以TXT的形式储存。

当真实数据被收集到后，它们往往不能直接使用。例如本次数据集使用了某地区的房价分布，为了简化模型数据只有两维，分别是房屋面积与房屋价格。可以看到房价与房屋面积之间存在一种关系，这种关系究竟是什么，就是本次预测想要得到的结论。可以首先以表格的形式输出数据的前五行看一下。

In [2]:
colnames = ['房屋面积']+['房价']
print_data = pd.read_csv('./data/data.txt',names = colnames)
print_data.head()

房屋面积    房价
0   79.47   750
1   83.79   850
2   59.82   285
3   55.10   205
4  144.00  1320

一般拿到一组数据后，第一个要处理的是数据类型不同的问题。如果各维属性中有离散值和连续值，就必须对离散值进行处理。

离散值虽然也常使用类似0、1、2这样的数字表示，但是其含义与连续值是不同的，因为这里的差值没有实际意义。例如，我们用0、1、2来分别表示红色、绿色和蓝色的话，我们并不能因此说“蓝色和红色”比“绿色和红色”的距离更远。通常对有d个可能取值的离散属性，我们会将它们转为d个取值为0或1的二值属性或者将每个可能取值映射为一个多维向量。

不过就这里而言，数据中没有离散值，就不用考虑这个问题了。

** 归一化 **

观察一下数据的分布特征，一般而言，如果样本有多个属性，那么各维属性的取值范围差异会很大，这就要用到一个常见的操作-归一化（normalization）了。归一化的目标是把各位属性的取值范围放缩到差不多的区间，例如[-0.5, 0.5]。这里我们使用一种很常见的操作方法：减掉均值，然后除以原取值范围。

虽然本次房价预测模型中，输入属性只有房屋面积，不存在取值范围差异问题，但由于归一化的各种优点，我们仍选择对其进行归一化操作。

** 练习 **

完成下列代码，实现房屋面积的归一化。

输出归一化后的房价最大值，判断操作是否正确。

In [3]:
# coding = utf-8 #
global x_raw,train_data,test_data
data = np.loadtxt('./data/data.txt',delimiter=',')
x_raw = data.T[0].copy() 

#axis=0,表示按列计算
#data.shape[0]表示data中一共有多少列
maximums,minimums,avgs = data.max(axis=0),data.min(axis=0),data.sum(axis=0)/data.shape[0]
print("the raw area :",data[:,0].max(axis = 0))

#归一化，data[:,i]表示第i列的元素

### START CODE HERE ### (≈ 3 lines of code)
feature_num = 2
for i in xrange(feature_num-1):
    data[:,i] = (data[:,i]-avgs[i])/(maximums[i]-minimums[i])
### END CODE HERE ###

print('normalization:',data[:,0].max(axis = 0))

the raw area : 451.0
normalization: 0.882389916071


期望输出：normalization: 0.8823899160713168

基本上所有的数据在拿到后都必须进行归一化，至少有以下3条原因：

1.过大或过小的数值范围会导致计算时的浮点上溢或下溢。

2.不同的数值范围会导致不同属性对模型的重要性不同（至少在训练的初始阶段如此），而这个隐含的假设常常是不合理的。这会对优化的过程造成困难，使训练时间大大加长。

3.很多的机器学习技巧/模型（例如L1，L2正则项，向量空间模型-Vector Space Model）都基于这样的假设：所有的属性取值都差不多是以0为均值且取值范围相近的。

** 数据集分割 **

将原始数据处理为可用数据后，为了评估模型的好坏，我们将数据分成两份：训练集和测试集。
- 训练集数据用于调整模型的参数，即进行模型的训练，模型在这份数据集上的误差被称为训练误差；
- 测试集数据被用来测试，模型在这份数据集上的误差被称为测试误差。

我们训练模型的目的是为了通过从训练数据中找到规律来预测未知的新数据，所以测试误差是更能反映模型表现的指标。分割数据的比例要考虑到两个因素：更多的训练数据会降低参数估计的方差，从而得到更可信的模型；而更多的测试数据会降低测试误差的方差，从而得到更可信的测试误差。我们这个例子中设置的分割比例为8:2。

** 练习 **

补全下列代码，完成训练集与测试集的分割。输出数据大小，验证分割是否正确。

- 提示：

    A[:x] 表示取A中的前x个元素
    
    A[x:]表示从第x+1个元素开始取到最后一个元素

In [4]:
ratio = 0.8
offset = int(data.shape[0]*ratio)

### START CODE HERE ### (≈ 2 lines of code)
train_data = data[:offset].copy()
test_data = data[offset:].copy()
### END CODE HERE ###

print('数据总个数：',len(data))
print('训练集数据个数:',len(train_data))

数据总个数： 2100
训练集数据个数: 1680


期望输出：

数据总个数：2100

训练集数据个数：1680

# 3 - 构造reader

**本部分完成数据读取reader的构造**

首先来介绍一下paddlepaddle的reader机制：

- **作用**：构造train_reader()函数，来读取训练数据集train_set或者测试数据集test_set。
- **用法**：具体实现是在read_data()函数内部构造一个reader()，使用yield关键字来让reader()成为一个Generator（生成器），注意，yield关键字的作用和使用方法类似return关键字，不同之处在于yield关键字可以构造生成器（Generator）。
- **优势**：虽然我们可以直接创建一个包含所有数据的列表，但是由于内存限制，我们不可能创建一个无限大的或者巨大的列表，并且很多时候在创建了一个百万数量级别的列表之后，我们却只需要用到开头的几个或几十个数据，这样造成了极大的浪费，而生成器的工作方式是在每次循环时计算下一个值，不断推算出后续的元素，不会创建完整的数据集列表，从而节约了内存使用。

关于Reader机制更加详细的介绍，可以参看此博客：https://blog.csdn.net/lizh_2018/article/details/79854117

In [5]:
def read_data(data_set):
    """
    一个reader
    Args：
        data_set -- 要获取的数据集
    Return：
        reader -- 用于获取训练集及其标签的生成器generator
    """
    def reader():
        """
        一个reader
        Args：
        Return：
            data[:-1],data[-1:] --使用yield返回生成器
                data[:-1]表示前n-1个元素，也就是训练数据，
                data[-1:]表示最后一个元素，也就是对应的标签
        """
        for data in data_set:
            yield data[:-1],data[-1:]
    return reader

获取训练数据集和测试数据集

In [6]:
def train():
    """
    定义一个reader来获取训练数据集及其标签
    Args：
    Return：
        read_data -- 用于获取训练数据集及其标签的reader
    """
    global train_data
    return read_data(train_data)

def test():
    global test_data
    return read_data(test_data)

测试reader：

In [7]:
test_array = ([10,100],[20,200])
print("test_array for read_data:")
for value in read_data(test_array)():
    print(value)

test_array for read_data:
([10], [100])
([20], [200])


# 4 - 模型训练


完成了数据的预处理工作并构造了read_data()来读取数据，接下来将进入模型的训练过程，使用PaddlePaddle来定义构造可训练的线性回归模型，关键步骤如下：

1. 初始化

2. 模型配置
    - 配置网络结构
    - 定义损失函数cost
    - 创建parameters
    - 定义优化器optimizer

3. 模型训练

4. 预测

5. 绘制拟合图像

** 1.初始化 **

首先进行最基本的初始化操作，在PaddlePaddle中使用paddle.init(use_gpu=False, trainer_count=1)来进行初始化：

- use_gpu=False表示不使用gpu进行训练
- trainer_count=1表示仅使用一个训练器进行训练

In [8]:
# 初始化
# paddle.init(use_gpu=False, trainer_count=1)
paddle.init(use_gpu=True, trainer_count=4)

** 2. 模型配置 **

** 配置网络结构 **


线性回归的模型其实就是一个采用线性激活函数（linear activation）的全连接层（fully-connected layer，fc_layer），因此在Peddlepeddle中利用全连接层模型构造线性回归，这样一个全连接层就可以看做是一个简单的神经网络，只包含输入层和输出层即可。本次的模型由于只有一个影响参数，因此输入只含一个$X_0$。

<img src="images/FC.png" style="width:380px;height:180px;">

接下来就让我们利用PaddlePaddle提供的接口，搭建我们自己的网络吧！

** 输入层 **

我们可以用x = paddle.layer.data(name='x', type=paddle.data_type.dense_vector(1))来表示数据的一个输入层，其中名称为"x"，数据类型为一维的稠密向量。

** 输出层 **

用y_predict = paddle.layer.fc(input=x, size=1， act=paddle.activation.Linear()来表示输出层：其中paddle.layer.fc表示全连接层，input=x表示该层出入数据为x，size=1表示该层有一个神经元，act=paddle.activation.Linear()表示激活函数为Linear()。


** 标签层 **

用y = paddle.layer.data(name='y', type=paddle.data_type.dense_vector(1))来表示标签数据，名称为y，数据类型为一维的稠密向量

** 练习 ** 完成下面代码

In [9]:
# 输入层，paddle.layer.data表示数据层,name=’x’：名称为x,
# type=paddle.data_type.dense_vector(1)：数据类型为1维稠密向量

### START CODE HERE ### (≈ 1 lines of code)
x = paddle.layer.data(name='x', type=paddle.data_type.dense_vector(1))
### END CODE HERE ###


# 输出层，paddle.layer.fc表示全连接层，input=x: 该层输入数据为x
# size=1：神经元个数，act=paddle.activation.Linear()：激活函数为Linear()

### START CODE HERE ### (≈ 1 lines of code)
y_predict = paddle.layer.fc(input=x, size=1, act=paddle.activation.Linear())
### END CODE HERE ###

    
    
# 标签数据，paddle.layer.data表示数据层，name=’y’：名称为y
# type=paddle.data_type.dense_vector(1)：数据类型为1维稠密向量

### START CODE HERE ### (≈ 1 lines of code)
y = paddle.layer.data(name='y', type=paddle.data_type.dense_vector(1))
### END CODE HERE ###

** 定义损失函数 **

本次使用的是均方差损失函数，可以调用paddle.layer.mes_cost(input= ,laybel= ) 实现。将输入定义为房价预测值，label定义为标签数据，计算损失值。

** 练习 ** 完成下面代码


In [10]:
# 定义成本函数为均方差损失函数square_error_cost

### START CODE HERE ### (≈ 1 lines of code)
cost = paddle.layer.square_error_cost(input=y_predict, label=y)
### END CODE HERE ###

** 创建parameters **

PaddlePaddle中提供了接口paddle.parameters.create(cost)来创建和初始化参数，参数cost表示基于我们刚刚创建的cost损失函数来创建和初始化参数。

In [11]:
# 利用cost创建parameters
parameters = paddle.parameters.create(cost)

** optimizer **

参数创建完成后，定义参数优化器optimizer= paddle.optimizer.Momentum(momentum=0)，使用Momentum作为优化器，并设置动量momentum为零。注意，读者暂时无需了解Momentum的含义，只需要学会使用即可。如果想要自己设置学习率，可以在paddle.optimizer.Momentum()中，自己添加参数。

** 练习 **

请自行设置学习率的值，观察cost函数的变化。

- 学习率参数为：learning_rate

In [12]:
# 创建optimizer，并初始化momentum, momentum=0为普通SGD优化算法
optimizer = paddle.optimizer.Momentum(momentum=0)

# 设置学习率参数
### START CODE HERE ### (≈ 1 lines of code)
#optimizer = paddle.optimizer.Momentum(momentum=0,learning_rate = 0.001)
### END CODE HERE ###


** 其它配置 **

feeding={‘image’:0, ‘label’:1}是数据层名称和数组索引的映射，用于在训练时输入数据，costs数组用于存储cost值，记录成本变化情况。
最后定义函数event_handler(event)用于事件处理，事件event中包含batch_id，pass_id，cost等信息，我们可以打印这些信息或作其它操作。

In [13]:
# 数据层和数组索引映射，用于trainer训练时喂数据
feeding = {'x': 0, 'y': 1}
    
# 用costs列表记录成本cost的值的变化
costs = []
#事件处理
def event_handler(event):
    """
    事件处理器，可以根据训练过程的信息作相应操作
    Args:
        event: 事件对象，包含event.pass_id, event.batch_id, event.cost等信息
    Return:
    """
    if isinstance(event, paddle.event.EndIteration):
        if event.pass_id % 100 == 0:
            print("Pass %d, Batch %d, Cost %f" % (
                event.pass_id, event.batch_id, event.cost))
            costs.append(event.cost)

    if isinstance(event, paddle.event.EndPass):
        result = trainer.test(
            reader=paddle.batch(test(), batch_size=2),
            feeding=feeding)
        print("Test %d, Cost %f" % (event.pass_id, result.cost))

** 3.  模型训练 **

上述内容进行了模型初始化、网络结构的配置并创建了损失函数、参数、优化器，接下来利用上述配置进行模型训练。


首先定义一个随机梯度下降trainer，配置三个参数cost、parameters、update_equation，它们分别表示损失函数、参数和更新公式。

再利用trainer.train()即可开始真正的模型训练，我们可以设置参数如下：

In [14]:
# 构造 trainer    
trainer = paddle.trainer.SGD(
cost=cost, parameters=parameters, update_equation=optimizer)

# 模型训练
trainer.train(
        reader=paddle.batch(
            paddle.reader.shuffle(train(), buf_size=500),
            batch_size=256),
        feeding=feeding,
        event_handler=event_handler,
        num_passes=300)

Pass 0, Batch 0, Cost 381930.000000
Pass 0, Batch 1, Cost 105661.687500
Pass 0, Batch 2, Cost 84693.046875
Pass 0, Batch 3, Cost 81217.664062
Pass 0, Batch 4, Cost 78127.281250
Pass 0, Batch 5, Cost 60165.437500
Pass 0, Batch 6, Cost 163684.972222
Test 0, Cost 141018.717317
Test 1, Cost 137334.331508
Test 2, Cost 134330.194560
Test 3, Cost 131214.761063
Test 4, Cost 128229.864706
Test 5, Cost 125311.075856
Test 6, Cost 122419.742337
Test 7, Cost 119833.745198
Test 8, Cost 117224.044799
Test 9, Cost 114837.262340
Test 10, Cost 112510.253550
Test 11, Cost 110079.003652
Test 12, Cost 108016.449871
Test 13, Cost 105782.499692
Test 14, Cost 103585.295756
Test 15, Cost 101573.138307
Test 16, Cost 99909.009071
Test 17, Cost 97828.089170
Test 18, Cost 96179.128656
Test 19, Cost 94669.975999
Test 20, Cost 92762.257148
Test 21, Cost 91034.680648
Test 22, Cost 89516.764147
Test 23, Cost 88151.522341
Test 24, Cost 86530.297209
Test 25, Cost 85033.472582
Test 26, Cost 83872.800402
Test 27, Cost 824

Test 270, Cost 42283.645320
Test 271, Cost 42251.094890
Test 272, Cost 42266.671309
Test 273, Cost 42271.978345
Test 274, Cost 42243.031193
Test 275, Cost 42285.438991
Test 276, Cost 42237.399772
Test 277, Cost 42249.226360
Test 278, Cost 42243.892704
Test 279, Cost 42260.002290
Test 280, Cost 42238.721971
Test 281, Cost 42258.622928
Test 282, Cost 42251.707958
Test 283, Cost 42233.486038
Test 284, Cost 42236.244517
Test 285, Cost 42223.292260
Test 286, Cost 42247.188926
Test 287, Cost 42245.839631
Test 288, Cost 42253.528121
Test 289, Cost 42222.663622
Test 290, Cost 42248.697383
Test 291, Cost 42229.095723
Test 292, Cost 42246.550368
Test 293, Cost 42215.903400
Test 294, Cost 42227.893267
Test 295, Cost 42219.181996
Test 296, Cost 42210.607781
Test 297, Cost 42230.093060
Test 298, Cost 42231.848660
Test 299, Cost 42198.440854


关于参数的解释如下：

- paddle.reader.shuffle(train(), buf_size=500)表示trainer从train()这个reader中读取了buf_size=500大小的数据并打乱顺序
- paddle.batch(reader(), batch_size=256)表示从打乱的数据中再取出batch_size=256大小的数据进行一次迭代训练
- 参数feeding用到了之前定义的feeding索引，将数据层image和label输入trainer，也就是训练数据的来源。
- 参数event_handler是事件管理机制，读者可以自定义event_handler，根据事件信息作相应的操作。
- 参数num_passes=300表示迭代训练300次后停止训练。

** 练习 **

同学们可以试着调整一下参数值，看会有什么变化。

In [15]:
# 构造 trainer    
trainer = paddle.trainer.SGD(
cost=cost, parameters=parameters, update_equation=optimizer)

# 模型训练
trainer.train(
        reader=paddle.batch(
            paddle.reader.shuffle(train(), buf_size=300 ),
            batch_size=128),
        feeding=feeding,
        event_handler=event_handler,
        num_passes=100)

Pass 0, Batch 0, Cost 356850.750000
Pass 0, Batch 1, Cost 253726.218750
Pass 0, Batch 2, Cost 114385.453125
Pass 0, Batch 3, Cost 105905.804688
Pass 0, Batch 4, Cost 109696.335938
Pass 0, Batch 5, Cost 78180.468750
Pass 0, Batch 6, Cost 106539.031250
Pass 0, Batch 7, Cost 42174.382812
Pass 0, Batch 8, Cost 61036.105469
Pass 0, Batch 9, Cost 83041.625000
Pass 0, Batch 10, Cost 71193.718750
Pass 0, Batch 11, Cost 129551.898438
Pass 0, Batch 12, Cost 126780.640625
Pass 0, Batch 13, Cost 122352.992188
Test 0, Cost 141359.033959
Test 1, Cost 137401.061870
Test 2, Cost 134167.738007
Test 3, Cost 131090.943564
Test 4, Cost 128157.335388
Test 5, Cost 125378.800131
Test 6, Cost 122620.498293
Test 7, Cost 119973.773291
Test 8, Cost 117318.282506
Test 9, Cost 114771.462566
Test 10, Cost 112450.070567
Test 11, Cost 110022.693572
Test 12, Cost 107989.652475
Test 13, Cost 105778.162542
Test 14, Cost 103758.208380
Test 15, Cost 101680.018504
Test 16, Cost 99789.043709
Test 17, Cost 97850.340802
Test 

** 4. 预测房价 **

训练结束后，输出回归模型的系数 a,b 和损失函数变化情况。通过观察损失函数变化，判断模型参数设置是否合理。

** 练习 **

根据线性模型的原理，补全输出公式，计算a和b的值


In [16]:
def print_parameters(parameters):
    """
        打印训练结果的参数以及测试结果
        Args:
            parameters -- 训练结果的参数
        Return:
        """
    print("Result Parameters as below:")
    a = parameters.get('___fc_layer_0__.w0')[0]
    b = parameters.get('___fc_layer_0__.wbias')[0]
    print(a, b)

    x0 = x_raw[0]
    y0 = a * train_data[0][0] + b

    x1 = x_raw[1]
    y1 = a * train_data[1][0] + b

    ### START CODE HERE ### (≈ 2 lines of code)
    a = (y0 - y1) / (x0 - x1)
    b = (y1 - a * x1)
    ### END CODE HERE ###

    print('a = ', a)
    print('b = ', b)
    
print_parameters(parameters)

Result Parameters as below:
3020.8 473.462
a =  7.07249417245
b =  -50.7058890577


预测结果应为：a=7.07,b=-54.56(在不同的机器上训练，得到的结果可能会有一点点偏差，但大致应在这个范围之间）,因此本次模型得到的房屋面积与房价之间的拟合函数为$y=7.07x-54.56$。其中y为预测的房屋价格，x为房屋面积，根据这个公式可以推断：如果有500万的预算，想在该地区购房，房屋面积大概为$\frac{500+(-54.56)}{7.07}=63(m^2)$。

In [17]:
def plot_costs(costs):
    """
    利用costs展示模型的训练曲线
    Args:
        costs -- 记录了训练过程的cost变化的list，每一百次迭代记录一次
    Return:
    """
    costs = np.squeeze(costs)
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("House Price Distributions ")
    plt.savefig('costs1.png')
    plt.show()
    
plot_costs(costs)

得到的cost函数变化图像大致应是：
<img src='images/result.png' style = "width:400px;height:300px;">

** 5. 绘制拟合图像  **

通过训练，本次线性回归模型输出了一条拟合的直线，想要直观的判断模型好坏可将拟合直线与数据的图像绘制出来。

**练习**

补全下列代码，绘制出你的预测图像（a和b的值应是你经过网络训练出来的结果）。

In [18]:
import numpy as np
import matplotlib.pyplot as plt

def plot_data(data,a,b):
    x = data[:,0]
    y = data[:,1]
    y_predict = x*a + b;
    plt.scatter(x,y,marker='.',c='r',label='True')
    plt.title('House Price Distributions')
    plt.xlabel('House Area ')
    plt.ylabel('House Price ')
    plt.xlim(0,500)
    plt.ylim(0,2500)
    predict = plt.plot(x,y_predict,label='Predict')
    plt.legend(loc='upper left')
    plt.savefig('result1.png')
    plt.show()

### START CODE HERE ### (≈ 2 lines of code)   
data = np.loadtxt('./data/data.txt', delimiter=',')
plot_data(data,7.07,-54.56)  
### END CODE HERE ###

从输出结果图可以看出，预测数据落在直线上，通过观察可以清楚地看到真实数据大部分散布在预测数据周围，说明预测结果是比较可靠的。

你得到的预测图，应与下图相似：<img src='images/plt.png' style = "width:400px;height:300px;">

# 5 - 总结

通过这个练习我们应该记住：

1. 机器学习的典型过程：
    
    - 获取数据
    
    - 数据预处理
    
    -训练模型
    
    -应用模型

2. PaddlePaddle训练模型的基本步骤: 

    - 初始化

    - 配置网络结构和设置参数：
        - 定义成本函数cost
        - 创建parameters
        - 定义优化器optimizer
    - 定义event_handler

    - 定义trainer

    - 开始训练

    - 预测infer()并输出准确率train_accuracy和test_accuracy

3. 练习中的许多参数可以作调整，例如修改学习率会对模型结果产生很大影响，大家可以在本练习或者后面的练习中多做些尝试。


至此线性回归模型的训练工作完成，希望通过本次课程的学习，读者可以利用提供的代码完成一个简单的房价预测模型。通过这一过程，初步了解PaddlePaddle这一易学易用的分布式平台。

本节课作为PaddlePaddle的快速入门章节，希望可以开启您的下一步深度学习之门。